In [37]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics

In [38]:
using Pkg
Pkg.add("PyCall")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [284]:
n_stations_test=5

5

In [285]:
# Read the data
stations_info = CSV.read("../data/stations/station_information.csv", Tables.matrix, header=1);
#stations_info

449×15 Matrix{Any}:
   3  "classic"  …  "B32006"  42.3401   10
   4  "classic"     "C32000"  42.3454   10
   5  "classic"     "B32012"  42.3418   10
   6  "classic"     "D32000"  42.3613   10
   7  "classic"     "A32000"  42.3534   10
   8  "classic"  …  "A32001"  42.3533   10
   9  "classic"     "A32002"  42.3517   10
  10  "classic"     "A32003"  42.3504   10
  11  "classic"     "A32004"  42.3386   10
  12  "classic"     "B32002"  42.3362   10
   ⋮             ⋱                     
 575  "classic"  …  "T32014"  42.5209  136
 576  "classic"     "F32001"  42.4117  139
 577  "classic"     "F32002"  42.4177  139
 578  "classic"     "F32003"  42.4017  139
 582  "classic"     "M32081"  42.3731    8
 583  "classic"  …  "M32082"  42.363     8
 584  "classic"     "G32001"  42.4265  140
 585  "classic"     "G32002"  42.4239  140
 586  "classic"     "V32015"  42.4082  104

In [286]:
n_stations=length(stations_info[:,1])

449

## Let's build our input data matrices

In [287]:
# Capacities C_i
C=stations_info[:,9][:,:];
C_test=C[1:n_stations_test,:];

In [288]:
# Distances D_ij
D=Array{Float64}(undef,n_stations,n_stations)
for i=1:n_stations
    for j=1:n_stations
        D[i,j]=sqrt((stations_info[i,6]-stations_info[j,6])^2+(stations_info[i,14]-stations_info[j,14])^2)
    end
end
D_test=D[1:n_stations_test,1:n_stations_test]

5×5 Matrix{Float64}:
 0.0        0.0314487  0.0105772  0.041174   0.0575982
 0.0314487  0.0        0.020872   0.0164451  0.0262908
 0.0105772  0.020872   0.0        0.0315853  0.0470539
 0.041174   0.0164451  0.0315853  0.0        0.0221594
 0.0575982  0.0262908  0.0470539  0.0221594  0.0

In [289]:
# Feasibility X_ij
threshold=0.01
X=Array{Float64}(undef,n_stations,n_stations)
for i=1:n_stations
    for j=1:n_stations
        if D[i,j]<threshold
            X[i,j]=1
        else
            X[i,j]=0
        end
    end
end
#X
X_test=X[1:n_stations_test,1:n_stations_test]

5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [290]:
# Let's count the non-zero values in X
sum(X[i,12] > 0 for i in 1:n_stations)

13

In [291]:
# Demand d_ijt
using PyCall

py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

d = load_pickle("../data/parameters/demand_matrix.pkl");

In [292]:
d_test=d[1:n_stations_test,1:n_stations_test,:]

5×5×744 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

;;; … 

[:, :, 742] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 743] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 744] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [295]:
sum(d_test[:,1,:])

89.0

In [272]:
# Number of vans
K=2
# Capacity of vans
S=10

10

In [273]:
# Initial number of bikes available y_i1
y1=Array{Int64}(undef,n_stations)
for i in 1:n_stations
    y1[i]=floor(C[i]/2)
end
y1_test=y1[1:n_stations_test];

## Simplified formulation

In [293]:
function first_model(C,D,X,y1,K,S,d,n_stations)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 100)
    M=30
    lambda=0.01 # to be tuned
    n_rebal_stations = 5
    # Decision variables
    @variable(model, x[1:n_rebal_stations, 1:n_rebal_stations, 1:K, 1:24], Bin)
    @variable(model, 0 <= z[1:n_rebal_stations, 1:n_rebal_stations, 1:K, 1:24], Int)
    @variable(model, 0 <= y[1:n_stations, 1:24]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:24], Int) # Removed Int
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:24])

    # Add constraints
    @constraint(model, [i in 1:n_stations, t in 1:24], y[i,t] <= C[i])
    @constraint(model, [i in 1:n_rebal_stations, k in 1:K, t in 1:24], x[i,i,k,t] == 0)
    @constraint(model, [i in 1:n_rebal_stations, t in 2:24], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)+sum(z[i,j,k,t] for j in 1:n_rebal_stations, k in 1:K))
    @constraint(model, [i in n_rebal_stations+1:n_stations, t in 2:24], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations], y[i,1] == y1[i])
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 2:24], z[i,j,k,t] <= S)
    @constraint(model, [i in 1:n_stations, t in 2:24], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 2:24], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    @constraint(model, [k in 1:K, t in 2:24], sum(x[i,j,k,t] for i in 1:n_rebal_stations, j in 1:n_rebal_stations) == 1)
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 2:24], z[i,j,k,t] <= M*x[i,j,k,t])
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 2:24], x[i,j,k,t] <= X[i,j])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 2:24], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    #@constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 2:24], u[i,j,t] >= w[i,j,t] - d[i,j,t])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:24)+lambda*sum(D[i,j]*x[i,j,k,t] for i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:24))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), value.(w), value.(u), objective_value(model)
end

first_model (generic function with 1 method)

In [283]:
x, w, u, obj=first_model(C_test,D_test,X_test,y1_test,K,S,d_test,n_stations_test)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter TimeLimit to value 100


MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [276]:
# count non zeros values in x
sum(x[i,j,k,t] > 0 for i in 1:3, j in 1:3, k in 1:K, t in 1:24)

0

In [277]:
sum(x[i,i,k,t] > 0 for i in 1:3, k in 1:K, t in 1:24)

0

In [278]:
sum(D[i,j]*x[i,j,k,t] for i in 1:3, j in 1:3, k in 1:K, t in 1:24)

0.0

In [279]:
D[1:3,1:3]

3×3 Matrix{Float64}:
 0.0        0.0314487  0.0105772
 0.0314487  0.0        0.020872
 0.0105772  0.020872   0.0

In [280]:
# count non zeros values in w
sum(w[i,j,t] > 0 for i in 1:n_stations_test, j in 1:n_stations_test, t in 1:24)

870

In [281]:
# list i,j for which u[i,j,t] is not null
a=[[i,j,t] for i in 1:n_stations_test, j in 1:n_stations_test, t in 1:24 if u[i,j,t] > 0]

Vector{Int64}[]

In [248]:
for i in 1:length(a)
    println(w[a[i][1],a[i][2],a[i][3]])
    println(d[a[i][1],a[i][2],a[i][3]])
    println(u[a[i][1],a[i][2],a[i][3]])
    print("\n")
end